In [1]:
import os

In [2]:
%pwd

'd:\\VS Code\\mydlproject\\Kidney_DL_project\\research'

In [3]:
os.chdir("../")

In [6]:
%pwd

'd:\\VS Code\\mydlproject\\Kidney_DL_project'

In [7]:
#entity

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int